# **Laboratorio 8: Ready, Set, Deploy! 👩‍🚀👨‍🚀**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Acuña U.
- Nombre de alumno 2: Martín Guzmán S.

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/sebastianacunau/MDS7202-Laboratorios-y-Proyecto)

## Temas a tratar

- Entrenamiento y registro de modelos usando MLFlow.
- Despliegue de modelo usando FastAPI
- Containerización del proyecto usando Docker

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar una solución a un problema a partir de ML
- Desplegar su solución usando MLFlow, FastAPI y Docker

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExODJnMHJzNzlkNmQweXoyY3ltbnZ2ZDlxY2c0aW5jcHNzeDNtOXBsdCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AbPdhwsMgjMjax5reo/giphy.gif" width="400">
</p>



Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔).

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir:

10. Potability (1 si es potable, 0 no potable)

Descripción de cada atributo se pueden encontrar en el siguiente link: [dataset](https://www.kaggle.com/adityakadiwal/water-potability)

# **1. Optimización de modelos con Optuna + MLFlow (2.0 puntos)**

El objetivo de esta sección es que ustedes puedan combinar Optuna con MLFlow para poder realizar la optimización de los hiperparámetros de sus modelos.

Como aún no hemos hablado nada sobre `MLFlow` cabe preguntarse: **¡¿Qué !"#@ es `MLflow`?!**

<p align="center">
  <img src="https://media.tenor.com/eusgDKT4smQAAAAC/matthew-perry-chandler-bing.gif" width="400">
</p>

## **MLFlow**

`MLflow` es una plataforma de código abierto que simplifica la gestión y seguimiento de proyectos de aprendizaje automático. Con sus herramientas, los desarrolladores pueden organizar, rastrear y comparar experimentos, además de registrar modelos y controlar versiones.

<p align="center">
  <img src="https://spark.apache.org/images/mlflow-logo.png" width="350">
</p>

Si bien esta plataforma cuenta con un gran número de herramientas y funcionalidades, en este laboratorio trabajaremos con dos:
1. **Runs**: Registro que constituye la información guardada tras la ejecución de un entrenamiento. Cada `run` tiene su propio run_id, el cual sirve como identificador para el entrenamiento en sí mismo. Dentro de cada `run` podremos acceder a información como los hiperparámetros utilizados, las métricas obtenidas, las librerías requeridas y hasta nos permite descargar el modelo entrenado.
2. **Experiments**: Se utilizan para agrupar y organizar diferentes ejecuciones de modelos (`runs`). En ese sentido, un experimento puede agrupar 1 o más `runs`. De esta manera, es posible también registrar métricas, parámetros y archivos (artefactos) asociados a cada experimento.

### **Todo bien pero entonces, ¿cómo se usa en la práctica `MLflow`?**

Es sencillo! Considerando un problema de machine learning genérico, podemos registrar la información relevante del entrenamiento ejecutando `mlflow.autolog()` antes entrenar nuestro modelo. Veamos este bonito ejemplo facilitado por los mismos creadores de `MLflow`:

```python
#!pip install mlflow
import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
```

Si ustedes ejecutan el código anterior en sus máquinas locales (desde un jupyter notebook por ejemplo) se darán cuenta que en su directorio *root* se ha creado la carpeta `mlruns`. Esta carpeta lleva el tracking de todos los entrenamientos ejecutados desde el directorio root (importante: si se cambian de directorio y vuelven a ejecutar el código anterior, se creará otra carpeta y no tendrán acceso al entrenamiento anterior). Para visualizar estos entrenamientos, `MLflow` nos facilita hermosa interfaz visual a la que podemos acceder ejecutando:

```
mlflow ui
```

y luego pinchando en la ruta http://127.0.0.1:5000 que nos retorna la terminal. Veamos en vivo algunas de sus funcionalidades!

<p align="center">
  <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZXVuM3A5MW1heDFpa21qbGlwN2pyc2VoNnZsMmRzODZxdnluemo2bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o84sq21TxDH6PyYms/giphy.gif" width="400">
</p>

Les dejamos también algunos comandos útiles:

- `mlflow.create_experiment("nombre_experimento")`: Les permite crear un nuevo experimento para agrupar entrenamientos
- `mlflow.log_metric("nombre_métrica", métrica)`: Les permite registrar una métrica *custom* bajo el nombre de "nombre_métrica"


## **1.1 Combinando Optuna + MLflow (2.0 puntos)**

Ahora que tenemos conocimiento de ambas herramientas, intentemos ahora combinarlas para **más sabor**. El objetivo de este apartado es simple: automatizar la optimización de los parámetros de nuestros modelos usando `Optuna` y registrando de forma automática cada resultado en `MLFlow`.

Considerando el objetivo planteado, se le pide completar la función `optimize_model`, la cual debe:
- **Optimizar los hiperparámetros del modelo `XGBoost` usando `Optuna`.**
- **Registrar cada entrenamiento en un experimento nuevo**, asegurándose de que la métrica `f1-score` se registre como `"valid_f1"`. No se deben guardar todos los experimentos en *Default*; en su lugar, cada `experiment` y `run` deben tener nombres interpretables, reconocibles y diferentes a los nombres por defecto (por ejemplo, para un run: "XGBoost con lr 0.1").
- **Guardar los gráficos de Optuna** dentro de una carpeta de artefactos de Mlflow llamada `/plots`.
- **Devolver el mejor modelo** usando la función `get_best_model` y serializarlo en el disco con `pickle.dump`. Luego, guardar el modelo en la carpeta `/models`.
- **Guardar el código en `optimize.py`**. La ejecución de `python optimize.py` debería ejecutar la función `optimize_model`.
- **Guardar las versiones de las librerías utilizadas** en el desarrollo.
- **Respalde las configuraciones del modelo final y la importancia de las variables** en un gráfico dentro de la carpeta `/plots` creada anteriormente.

*Hint: Le puede ser útil revisar los parámetros que recibe `mlflow.start_run`*

```python
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_f1")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model
```

### Paquetes


In [5]:
#!pip install mlflow
#!pip install optuna
#!pip install -U kaleido

### Codigo

In [2]:
%%writefile optimize.py
import pandas as pd
import mlflow
import pickle
import optuna
import os
import kaleido
import matplotlib.pyplot as plt
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from mlflow.exceptions import MlflowException
import mlflow.sklearn


def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_run = runs.sort_values("metrics.valid_f1", ascending=False).iloc[0]
    best_model_id = best_run["run_id"]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")
    return best_model


def optimize_model():
  df = pd.read_csv('water_potability.csv')
  le = LabelEncoder()
  y = le.fit_transform(df['Potability'])
  X = df.drop('Potability', axis=1)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


  mlflow.autolog()
  nombre_experimento = f"ESTUDIO DE MODELO DE XGBOOST CON OPTIMIZACION PARA POTABILIDAD DE AGUA "

  try: #CODIGO PARA CHEQUEAR SI EXPERIMENTO YA ESTA CREADO
      experimento = mlflow.get_experiment_by_name(nombre_experimento)
      if experimento is None:
          experimento_id = mlflow.create_experiment(nombre_experimento) #CREACION EXPERIMENTO
      else:
          experimento_id = experimento.experiment_id
  except MlflowException as e:
      experimento = mlflow.get_experiment_by_name(nombre_experimento)
      if experimento is None:
          raise e
      experimento_id = experimento.experiment_id



  def objetivo(trial):
    params = {
        "objective": "multi:softmax",
        "eval_metric": "mlogloss",
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "gamma": trial.suggest_float("gamma", 0, 1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 300),
        "num_class": len(le.classes_)
    }


#inicio run creacion modelo
    run_name = f" Trial de XGBoost # {trial.number}, con max_depth = {params['max_depth']:.4f}, learning_rate = {params['learning_rate']:.4f}"

    with mlflow.start_run(run_name = run_name, experiment_id = experimento_id, nested=True) as run:

        model = XGBClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        f1 = f1_score(y_test, y_pred)

        mlflow.log_metric("valid_f1", f1) #log de metric f1 score
        mlflow.sklearn.log_model(model, "model")


    return f1

  study = optuna.create_study(direction="maximize") #OPTIMIZACION CON OPTUNA
  study.optimize(objetivo, n_trials=25) # optimizacion maximizando con la funcion objetivo

#Punto 1 CHECK
#Punto 2 CHECK
  #Inicio run visualizacion y guardado de plots
  run_name1 = "Visualizacion de Optimizacion con Optuna" #inico 2do run
  with mlflow.start_run(run_name = run_name1, experiment_id = experimento_id):
    historial_optimizacion = plot_optimization_history(study) #CREACION GRAFICOS CON OPTUNA
    plot_coordenadas_paralelas = plot_parallel_coordinate(study)
    plot_importancia_parametros  = plot_param_importances(study)

    if not os.path.exists("plots"): #CREACION CARPETA PLOTS
      os.makedirs("plots")

    if 'kaleido' in globals() and kaleido:
        historial_optimizacion.write_image("plots/historial_optimizacion.png")
        plot_coordenadas_paralelas.write_image("plots/plot_coordenadas_paralelas.png")
        plot_importancia_parametros.write_image("plots/plot_importancia_parametros.png")

    mlflow.log_artifact("plots/historial_optimizacion.png", artifact_path="plots") #LOG DE IMAGENES
    mlflow.log_artifact("plots/plot_coordenadas_paralelas.png", artifact_path= "plots")
    mlflow.log_artifact("plots/plot_importancia_parametros.png", artifact_path= "plots")


#PUNTO 3 CHECK


  best_model = get_best_model(experimento_id)
  run_name2 = "Mejor Modelo" #Inicio 3er run
  with mlflow.start_run(run_name = run_name2, experiment_id = experimento_id):


    if not os.path.exists("models"): #CREACION DE DIRECTORIO MODELS
      os.makedirs("models")

    with open("models/best_model.pkl", "wb") as f:
      pickle.dump(best_model, f)

    mlflow.log_artifact("models/best_model.pkl", artifact_path="models") #LOG DEL MEJOR MODELO EN DIRECTORIO MODELS



#PUNTO 4 CHECK

    best_params = best_model.get_params()
    feature_importances = best_model.feature_importances_
    feature_names = X.columns
    feature_importances = pd.Series(feature_importances, index=feature_names)
    feature_importances.sort_values(inplace=True)
    plt.figure(figsize=(10, 6))
    plt.bar(feature_importances.index, feature_importances.values)
    plt.xlabel("Features")
    plt.ylabel("Importance")
    plt.title("Feature Importances")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig("plots/feature_importances.png")
    mlflow.log_artifact("plots/feature_importances.png", artifact_path="plots")

    with open("plots/best_model_params.txt", "w") as f:
      for key, value in best_params.items():
        f.write(f"{key}: {value}\n")
    mlflow.log_artifact("plots/best_model_params.txt", artifact_path="plots")

#PUNTO 7 CHECK

  return best_model


if __name__ == "__main__":
  optimize_model()

Writing optimize.py


In [3]:
!pip freeze > requirements.txt
#Punto 6 check

In [4]:
!python optimize.py

2025/06/04 00:01:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/06/04 00:01:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
[I 2025-06-04 00:01:59,395] A new study created in memory with name: no-name-b557f92d-689c-48b1-b73e-54580abce8ae
/Users/sebastianacunaurzua/Documents/GitHub/MDS7202-Laboratorios-y-Proyecto/Laboratorios/Lab 8 - MLOps/optimize.py:57: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
2025/06/04 00:02:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/sebastianacunaurzua/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [00:02:08] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1744329023465/work/src

# **2. FastAPI (2.0 puntos)**

<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

Con el modelo ya entrenado, la idea de esta sección es generar una API REST a la cual se le pueda hacer *requests* para así interactuar con su modelo. En particular, se le pide:

- Guardar el código de esta sección en el archivo `main.py`. Note que ejecutar `python main.py` debería levantar el servidor en el puerto por defecto.
- Defina `GET` con ruta tipo *home* que describa brevemente su modelo, el problema que intenta resolver, su entrada y salida.
- Defina un `POST` a la ruta `/potabilidad/` donde utilice su mejor optimizado para predecir si una medición de agua es o no potable. Por ejemplo, una llamada de esta ruta con un *body*:

```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}
```

Su servidor debería retornar una respuesta HTML con código 200 con:


```json
{
  "potabilidad": 0 # respuesta puede variar según el clasificador que entrenen
}
```

**`HINT:` Recuerde que puede utilizar [http://localhost:8000/docs](http://localhost:8000/docs) para hacer un `POST`.**

In [6]:
#!pip install "fastapi[all]"

In [ ]:
%%writefile main.py
from fastapi import FastAPI
import uvicorn
import pickle
import pandas as pd
from pydantic import BaseModel




class parametros(BaseModel): #DEFINICION DE LOS PARAMETROS DEL MODELO, PARA QUE SEAN UTILIZADOS EN EL POST DEL PROGRAMA
  ph: float
  Hardness: float
  Solids: float
  Chloramines: float
  Sulfate: float
  Conductivity: float
  Organic_carbon: float
  Trihalomethanes: float
  Turbidity: float

app = FastAPI() #CREACION DE APP


try:
  with open("models/best_model.pkl", "rb") as f:
    model = pickle.load(f)
except FileNotFoundError:
  model = None
  print("El archivo models/best_model.pkl no existe.")



@app.get("/")
async def home():
  return {
      "Mensaje": "Bievenid@s, esta es una API para predicción de potabilidad del agua",
      "Descripción": "Esta predicción usa un modelo XGBoost con optimización de hiperparametros desde optuna, este modelo utiliza distintos parametros del agua",
      "Input": {
          "tipo": "objecto JSON, base de datos con los siguientes parametros",
          "parametros":[
              "ph",
              "Hardness",
              "Solids",
              "Chloramines",
              "Sulfate",
              "Conductivity",
              "Organic_carbon",
              "Trihalomethanes",
              "Turbidity"
          ]},
      "Output": {
          "tipo": "Objeto JSON",
          "parametro": "potabilidad",
          "valores": "1 si es potable, 0 si no lo es"
          }}



@app.post("/potabilidad")
async def predecir_potabilidad(data: parametros):
  if model is None:
    return {"error": "El modelo no está disponible."}
  try:
    input_data = pd.DataFrame([data.dict()])
    prediction_potabilidad = model.predict(input_data)
    return {"potabilidad": int(prediction_potabilidad[0])}
  except Exception as e:
    return {"error": str(e)}


if __name__ == '__main__':
    uvicorn.run("main:app", host="0.0.0.0", port=8000) #se utiliza el puerto default 8000.


Writing main.py


In [8]:
!python main.py

INFO:     Started server process [41805]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
^C
INFO:     Finished server process [41805]
ERROR:    Traceback (most recent call last):
  File "/Users/sebastianacunaurzua/opt/anaconda3/lib/python3.12/asyncio/runners.py", line 194, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/Users/sebastianacunaurzua/opt/anaconda3/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/sebastianacunaurzua/opt/anaconda3/lib/python3.12/asyncio/base_events.py", line 672, in run_until_complete
    self.run_forever()
  File "/Users/sebastianacunaurzua/opt/anaconda3/lib/python3.12/asyncio/base_events.py", line 639, in run_forever
    self._run_once()
  File "/Users/sebastianacunaurzua/opt/anaconda3/lib/python3.12/asyncio/base_even

# **3. Docker (2 puntos)**

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*9rafh2W0rbRJIKJzqYc8yA.gif" width="500">
</div>

Tras el éxito de su aplicación web para generar la salida, Smapina le solicita que genere un contenedor para poder ejecutarla en cualquier computador de la empresa de agua potable.

## **3.1 Creación de Container (1 punto)**

Cree un Dockerfile que use una imagen base de Python, copie los archivos del proyecto e instale las dependencias desde un `requirements.txt`. Con esto, construya y ejecute el contenedor Docker para la API configurada anteriormente. Entregue el código fuente (incluyendo `main.py`, `requirements.txt`, y `Dockerfile`) y la imagen Docker de la aplicación. Para la dockerización, asegúrese de cumplir con los siguientes puntos:

1. **Generar un archivo `.dockerignore`** que ignore carpetas y archivos innecesarios dentro del contenedor.
2. **Configurar un volumen** que permita la persistencia de los datos en una ruta local del computador.
3. **Exponer el puerto** para acceder a la ruta de la API sin tener que entrar al contenedor directamente.
4. **Incluir imágenes en el notebook** que muestren la ejecución del contenedor y los resultados obtenidos.
5. **Revisar y comentar los recursos utilizados por el contenedor**. Analice si los contenedores son livianos en términos de recursos.

## **3.2 Preguntas de Smapina (1 punto)**
Tras haber experimentado con Docker, Smapina desea profundizar más en el tema y decide realizarle las siguientes consultas:

- ¿Cómo se diferencia Docker de una máquina virtual (VM)?
- ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?
- ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?
- ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?
- ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?

> ¿Cómo se diferencia Docker de una máquina virtual (VM)?

Docker es un software que por medio de los *contenedores* permite crear y ejecutar aplicaciones, con todas las dependencias y configuraciones necesarias para trabajar (como por ejemplo, especificaciones de otro sistema operativo), sin tener que generar particiones en el Hardware, como sí lo hace una VM. En palabras sencillas y de forma bastante holística, Docker permite "usar" otros sistemas operativos de manera liviana y de rápida migración, mientras que una VM requiere instalar el sistema operativo completo en una nueva partición (lo cual es muy pesado) para poder operar.

> ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?

Al usar Docker, la aplicación se ejecuta dentro de un contenedor con todas las especificaciones y dependencias compatibles para su funcionamiento y es replicable para cualquier equipo que tenga instalado Docker. En cambio, al ejecutar la aplicación directamente en el sistema local, si la aplicación fue creada en la misma máquina probablemente funcione pero puede fallar en otra que no tenga las mismas especificaciones: el clásico "pero en mi máquina funcionaba". Aún peor, si la aplicación no fue creada en la misma máquina que la que se está ejecutando, pueden presentarse dificultades de compatibilidad y conflictos de dependencias que obligan a instalar manualmente todas las dependencias requeridas para el funcionamiento de la aplicación... o bien instalar Docker.

> ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?

Por medio de la creación de *imágenes* y el uso de *contenedores*. Lo primero hace referencia a la creación del `Dockerfile` con todas las especificaciones base y las dependencias necesarias para el funcionamiento de la(s) aplicación(es) a ejecutar (sistema operativo, librerías, etc.) y lo segundo hace referencia a un cierto tipo de "entorno" en donde esta imagen es ejecutada y por lo tanto, las aplicaciones almacenadas en este contenedor funcionan bajo las especificaciones de dicha imagen. Con ello, se pueden tener diferentes contenedores que pueden interactuar entre sí y tener cada uno sus especificaciones para que los entornos de desarrollo y producción funcionen sin presentar discrepancias ni problemas en su ejecución.

> ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?

Los volúmenes se gestionan de manera independiente a los contenedores y permiten almacenar información importante que de otro modo se perdería en un contenedor luego de ser reiniciado o eliminado. En ese sentido, funcionan como una memoria caché dentro de Docker para almacenar información relevante cada vez que se solicite. Existen comandos para crear volúmenes y guardarlos en directorios específicos al momento de ejecutar un contenedor.

> ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?

* Dockerfile: Es un archivo de texto que contiene "instrucciones" para construir una imagen de Docker. Es decir, define las especificaciones del entorno de la aplicación a ejecutar: qué sistema base usar, qué dependencias instalar, cómo copiar archivos, qué comandos ejecutar, etc.

* docker-compose.yml: Es un archivo que permite definir y gestionar múltiples contenedores y sus relaciones (por ejemplo, una app, una base de datos, un cache). Define cómo se deben ejecutar los contenedores, sus variables de entorno, volúmenes, redes, dependencias, etc. Es muy útil cuando se tiene una aplicación (o varias) con muchas utilidades y servicios diferentes.

# Conclusión

Éxito!
<div align="center">
  <img src="https://i.pinimg.com/originals/55/f5/fd/55f5fdc9455989f8caf7fca7f93bd96a.gif" width="500">
</div>